In [1]:
!pip install ninja

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 108 kB 38.6 MB/s 


In [2]:
!git clone https://github.com/PeikeLi/Self-Correction-Human-Parsing
%cd Self-Correction-Human-Parsing
!mkdir checkpoints
!mkdir inputs
!mkdir outputs
!mkdir outputs_lip

Cloning into 'Self-Correction-Human-Parsing'...
remote: Enumerating objects: 719, done.
remote: Total 719 (delta 0), reused 0 (delta 0), pack-reused 719
Receiving objects: 100% (719/719), 3.80 MiB | 2.81 MiB/s, done.
Resolving deltas: 100% (141/141), done.
/content/Self-Correction-Human-Parsing


In [3]:
import gdown

url = 'https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH'
output = 'checkpoints/lip.pth'
gdown.download(url, output, quiet=False)
url = 'https://drive.google.com/uc?id=1ruJg4lqR_jgQPj-9K0PP-L2vJERYOxLP'
output = 'checkpoints/atr.pth'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH
To: /content/Self-Correction-Human-Parsing/checkpoints/lip.pth
100%|██████████| 267M/267M [00:01<00:00, 142MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1ruJg4lqR_jgQPj-9K0PP-L2vJERYOxLP
To: /content/Self-Correction-Human-Parsing/checkpoints/atr.pth
100%|██████████| 267M/267M [00:04<00:00, 59.1MB/s]


'checkpoints/atr.pth'

### 请先上传需要分割的图片至 *inputs* 中，再运行

In [6]:
!python3 simple_extractor.py --dataset 'lip' --model-restore 'checkpoints/lip.pth' --input-dir 'inputs' --output-dir 'outputs_lip'
!python3 simple_extractor.py --dataset 'atr' --model-restore 'checkpoints/atr.pth' --input-dir 'inputs' --output-dir 'outputs'

Evaluating total class number 20 with ['Background', 'Hat', 'Hair', 'Glove', 'Sunglasses', 'Upper-clothes', 'Dress', 'Coat', 'Socks', 'Pants', 'Jumpsuits', 'Scarf', 'Skirt', 'Face', 'Left-arm', 'Right-arm', 'Left-leg', 'Right-leg', 'Left-shoe', 'Right-shoe']
100% 1/1 [00:00<00:00,  1.68it/s]
Evaluating total class number 18 with ['Background', 'Hat', 'Hair', 'Sunglasses', 'Upper-clothes', 'Skirt', 'Pants', 'Dress', 'Belt', 'Left-shoe', 'Right-shoe', 'Face', 'Left-leg', 'Right-leg', 'Left-arm', 'Right-arm', 'Bag', 'Scarf']
100% 1/1 [00:00<00:00,  2.52it/s]


In [ ]:
!mkdir samples

import os
from PIL import Image

outputs = os.listdir('./outputs')


for f in outputs:
  if f.startswith('.'):
    continue
  img0 = Image.open('./outputs/' + f)			# ATR Image
  sample = Image.open('sample.png')				# Sample Image
  width, height = img0.size

  palette11 = sample.getpalette()

  load0 = img0.load()

  for i in range(width):						# 遍历所有像素进行颜色转换
    for n in range(height):
      if(load0[i, n] == 4):
        load0[i, n] = 5
      elif(load0[i, n] == 5):
        load0[i, n] = 12
      elif(load0[i, n] == 11):
        load0[i, n] = 13
      elif(load0[i, n] == 12):
        load0[i, n] = 16
      elif(load0[i, n] == 13):
        load0[i, n] = 17
  img0.putpalette(palette11)					# 将ATR Image的调色盘置为Sample Image的

  # -------------- add neck by brown color -------------- #
  small_face, big_face = [], []      # 储存着从不同照片中读取的脸部像素坐标
  no_neck_img = Image.open('./outputs_lip/' + f)      
  no_neck_load = no_neck_img.load()
  for i in range(width):                          # width = 768
    for n in range(height):                     # height = 1024
      if no_neck_load[i, n] == 13:			# pixel 13 means face
        small_face.append([i, n])			# 获取LIP输出图片中脸部的坐标
      if load0[i, n] == 13:
        big_face.append([i, n])				# 获取ATR输出图片中脸部的坐标


  for i in big_face:								# 存在在ATR脸部中但不在LIP脸部
    if i not in small_face:						# 中的像素就是脖子Neck的像素
          load0[i[0], i[1]] = 10					# 变成棕色,即Sample中脖子的颜色
          
  img0.save('./samples/' + f)

### 可以从 *outputs* 文件夹下载生成的结果图